In [9]:
import torch
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import Settings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core.indices.service_context import ServiceContext
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt,completion_to_prompt
from llama_index.core.node_parser import SentenceSplitter

In [10]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]  = ""

In [11]:
documents = SimpleDirectoryReader("../pdfs/").load_data()

In [16]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    #You can also use others LLMs of bigger size by using Quatisation through bitsandbytes
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

In [13]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
)

In [14]:
Settings.llm = llm
Settings.text_splitter = SentenceSplitter(chunk_size=256,chunk_overlap=20)
Settings.embed_model = embed_model

In [17]:
index = VectorStoreIndex.from_documents(documents)

In [20]:
query_engine = index.as_query_engine()
response = query_engine.query("What are pdfs given to you as input about?")

Llama.generate: prefix-match hit


In [21]:
print(response)

 The pdfs provided to me as input seem to be technical reports related to Amazon Web Services (AWS). The first pdf, "awsgsg-intro.pdf", appears to be an introduction to AWS services for beginners. The second pdf, "Final_Report (1).pdf", appears to be a technical report discussing IP problems and their solutions.
